In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import *
import html

In [3]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH=Path('../../data/fr-wiki/french')

# Standarize format

In [5]:
LANG_FILENAMES = [str(f) for f in PATH.rglob("*/*")]

In [7]:
len(LANG_FILENAMES), LANG_FILENAMES[:5]

(3906,
 ['..\\..\\data\\fr-wiki\\french\\AA\\wiki_00',
  '..\\..\\data\\fr-wiki\\french\\AA\\wiki_01',
  '..\\..\\data\\fr-wiki\\french\\AA\\wiki_02',
  '..\\..\\data\\fr-wiki\\french\\AA\\wiki_03',
  '..\\..\\data\\fr-wiki\\french\\AA\\wiki_04'])

In [11]:
LANG_TEXT = []
for fn in tqdm(LANG_FILENAMES):
    for line in open(fn, encoding='utf8'):
        LANG_TEXT.append(json.loads(line))
        
LANG_TEXT = pd.DataFrame(LANG_TEXT)

100%|██████████████████████████████████████████████████████████████████████████████| 3906/3906 [01:59<00:00, 32.67it/s]


In [12]:
LANG_TEXT.head()

,id,text,title,url
0,3,"Antoine Meillet\n\nPaul Jules Antoine Meillet,...",Antoine Meillet,https://fr.wikipedia.org/wiki?curid=3
1,7,Algèbre linéaire\n\nL’algèbre linéaire est la ...,Algèbre linéaire,https://fr.wikipedia.org/wiki?curid=7
2,9,"Algèbre générale\n\nL'algèbre générale, ou alg...",Algèbre générale,https://fr.wikipedia.org/wiki?curid=9
3,10,Algorithmique\n\nLalgorithmique est l'étude et...,Algorithmique,https://fr.wikipedia.org/wiki?curid=10
4,11,Politique en Argentine\n\nL'Argentine est une ...,Politique en Argentine,https://fr.wikipedia.org/wiki?curid=11


In [13]:
LANG_TEXT.to_csv(PATH/'wiki_fr.csv')

In [14]:
# Getting rid of the title name in the text field
def split_title_from_text(text):
    words = text.split("\n\n")
    if len(words) >= 2:
        return ''.join(words[1:])
    else:
        return ''.join(words)
    
LANG_TEXT['text'] = LANG_TEXT['text'].apply(lambda x: split_title_from_text(x))

In [15]:
LANG_TEXT.head()

,id,text,title,url
0,3,"Paul Jules Antoine Meillet, né le à Moulins (A...",Antoine Meillet,https://fr.wikipedia.org/wiki?curid=3
1,7,L’algèbre linéaire est la branche des mathémat...,Algèbre linéaire,https://fr.wikipedia.org/wiki?curid=7
2,9,"L'algèbre générale, ou algèbre abstraite, est ...",Algèbre générale,https://fr.wikipedia.org/wiki?curid=9
3,10,Lalgorithmique est l'étude et la production de...,Algorithmique,https://fr.wikipedia.org/wiki?curid=10
4,11,L'Argentine est une république présidentielle ...,Politique en Argentine,https://fr.wikipedia.org/wiki?curid=11


In [19]:
LANG_TEXT.to_csv(PATH/'wiki_fr1.csv', header=False, index=False)

Sorting the articles by length and keeping the first million.

In [11]:
LANG_TEXT = pd.read_csv(PATH/'wiki_fr1.csv', header=None)

In [17]:
LANG_TEXT = LANG_TEXT.assign(length = 0)

In [18]:
LANG_TEXT.columns = ['id', 'text', 'title', 'url', 'length']

In [20]:
LANG_TEXT = LANG_TEXT.assign(labels = 0).pipe(lambda x: x[['labels', 'text', 'length']])

In [21]:
LANG_TEXT.head()

,labels,text,length
0,0,"Paul Jules Antoine Meillet, né le à Moulins (A...",0
1,0,L’algèbre linéaire est la branche des mathémat...,0
2,0,"L'algèbre générale, ou algèbre abstraite, est ...",0
3,0,Lalgorithmique est l'étude et la production de...,0
4,0,L'Argentine est une république présidentielle ...,0


In [25]:
LANG_TEXT['length'] = LANG_TEXT['text'].str.len()

In [29]:
LANG_TEXT = LANG_TEXT.sort_values(by=['length'], ascending=False)

In [38]:
LANG_TEXT.to_csv(PATH/'wiki_fr1.csv', header=False, index=False)

In [41]:
LANG_TEXT = LANG_TEXT[LANG_TEXT['length'] > 100]

In [46]:
LANG_TEXT = LANG_TEXT.iloc[0:1000000]

Splitting 10% for validation.

In [49]:
trn_texts,val_texts = sklearn.model_selection.train_test_split(LANG_TEXT.pipe(lambda x: x[['labels', 'text']]), test_size=0.1)

In [52]:
trn_texts.to_csv(PATH/'train.csv', header=False, index=False)
val_texts.to_csv(PATH/'valid.csv', header=False, index=False)

# Tokenize

In [4]:
chunksize = 5000

In [5]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [6]:
df_trn = pd.read_csv(PATH/'train.csv', header=None, chunksize=chunksize)
df_val = pd.read_csv(PATH/'valid.csv', header=None, chunksize=chunksize)

In [7]:
def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = texts.apply(fixup).values.astype(str)
    tok = Tokenizer.proc_all_mp(partition_by_cores(texts), lang='fr')
    return tok, list(labels)

def get_all(df, name, n_lbls=1):
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        #save the partial tokens instead of regrouping them in one big array.
        np.save(PATH/f'{name}_tok{i}.npy', tok_)
    return tok, labels

In [ ]:
get_all(df_trn,'trn',1)

In [ ]:
get_all(df_val,'val',1)

# Numericalize

Get the Counter object from all the splitted files.

In [10]:
def count_them_all(names):
    cnt = Counter()
    for name in names:
        for file in PATH.glob(f'tmp/{name}_tok*'):
            tok = np.load(file)
            cnt_tok = Counter(word for sent in tok for word in sent)
            cnt += cnt_tok
    return cnt

In [11]:
cnt = count_them_all(['trn'])

In [13]:
cnt.most_common()

[(',', 404771),
 ('de', 403453),
 ('.', 240390),
 ('la', 229440),
 ('le', 174666),
 ('et', 165450),
 ('à', 143193),
 ('en', 131044),
 ("l'", 120337),
 ('les', 110875),
 ('"', 110053),
 ('des', 105044),
 ('du', 95649),
 ('est', 91542),
 ("d'", 84666),
 ('un', 75756),
 ('une', 68106),
 ('il', 65473),
 ('\r\r\n', 64888),
 ('(', 59492),
 ('dans', 59185),
 (')', 54383),
 ('par', 53859),
 ('au', 51115),
 ('-', 49281),
 ('pour', 46147),
 ('qui', 39651),
 ('a', 34453),
 ('sur', 33718),
 ('que', 33183),
 ('son', 30532),
 ('t_up', 30484),
 ('avec', 29857),
 ('se', 25568),
 ('plus', 25213),
 (':', 24598),
 ('l’', 22258),
 ('sont', 22242),
 ('elle', 19998),
 ('sa', 19070),
 ('ce', 18273),
 ('ou', 18171),
 ('«', 17461),
 ('»', 17358),
 ("s'", 16638),
 ('aux', 16534),
 ('été', 15906),
 ('ses', 15597),
 ("qu'", 15529),
 ('comme', 15440),
 ('1', 15148),
 ('d’', 14818),
 ('deux', 14337),
 ('cette', 14224),
 ('mais', 13323),
 ('pas', 13032),
 ('ne', 12358),
 ('\n', 12050),
 ('xbos', 12000),
 ('xfld', 12

In [14]:
max_vocab = 60000
min_freq = 5

In [16]:
itos = [o for o,c in cnt.most_common(max_vocab) if c > min_freq]
itos.insert(0,'_pad_')
itos.insert(0,'_unk_')

In [17]:
len(itos)

48446

In [18]:
stoi = collections.defaultdict(int,{s:i for (i,s) in enumerate(itos)})

Numericalize each partial file.

In [30]:
def numericalize(name):
    results = []
    for file in tqdm(PATH.glob(f'tmp/{name}_tok*')):
        tok = np.load(file)
        results.append(np.array([[stoi[word] for word in sent] for sent in tok]))
    return np.concatenate(results)

In [ ]:
trn_ids = numericalize('trn')
np.save(PATH/'tmp/trn_ids.npy', trn_ids)

In [ ]:
val_ids = numericalize('val')
np.save(PATH/'tmp/trn_ids.npy', val_ids)